<a href="https://colab.research.google.com/github/prasadanvekar/All-India-Pincode-Directory/blob/master/Data_upsert_pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pinecone-client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.1 MB/s eta 0:00:00


In [1]:
%env PINECONE_API_KEY=

env: PINECONE_API_KEY=


In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec
import csv
import numpy as np
from sklearn.preprocessing import normalize

# Initialize the Pinecone client
#pinecone.init(api_key="pcsk_6r6Hde_K6yg7eqVznKda55HScGrZuBv2P17tcm9jaY9Rxf566Ttn5uMxF7Zcxv4kgXA3DM", environment="us-east-1-aws")  # Replace with your Pinecone API key and environment
PINECONE_API_KEY=""

pc = Pinecone(
    api_key=os.environ.get("PINECONE_API_KEY"),
)

index_name="shopwise-order-data"

# **Change 1: Set the correct dimension (3072) during index creation**
dimension = 3072  # Update this to the actual dimension of your vectors

# Create the index if it doesn't exist
if index_name not in pc.list_indexes().names():
    pc.create_index(name= index_name, dimension=dimension
                    , metric="consine"
                    , spec=ServerlessSpec(
                        cloud='aws'
                        ,region ='us-east-1'
                    )
                  )  # Adjust dimension according to your vector size

def read_csv(file_path):
    ids = []
    vectors = []
    metadata = []
    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header row
        for row in reader:
            id = row[0]  # Assuming first column is the ID
            #vector = np.array([float(x) for x in row[1:-1] if x.replace('.', '', 1).isdigit()])
            #meta = ' '.join([x for x in row[1:-1] if not x.replace('.', '', 1).isdigit()])  # Updated to include potential non-numeric data
            # Change 2: Adjust vector extraction to ensure 3072 dimensions
            #vector = np.array([float(x) for x in row[1:] if x.replace('.', '', 1).isdigit()])
            #meta = ' '.join([x for x in row[1:] if not x.replace('.', '', 1).isdigit()])
            #meta = row[-1]  # Last column is metadata
            #vector = np.array([float(x) for x in row[1:-1]])  # Assuming the vector is in the columns before the last
            #meta = row[-1]  # Last column is metadata

            # **Change:** Extract all numeric values after the ID as the vector
            vector = np.array([float(x) for x in row[1:] if x.replace('.', '', 1).isdigit()])
            # **Change:** If vector has less than 3072 dimensions, pad with zeros
            if len(vector) < 3072:
                vector = np.pad(vector, (0, 3072 - len(vector)), 'constant')
            # **Change:** Extract all non-numeric values after the ID as the metadata
            meta = ' '.join([x for x in row[1:] if not x.replace('.', '', 1).isdigit()])

            # **Change:** Truncate metadata to a maximum length of 500 characters
            meta = meta[:250]  # Truncate metadata


            ids.append(id)
            vectors.append(vector)
            metadata.append(meta)
    return ids, vectors, metadata

# Optional: Normalize vectors
def normalize_vectors(vectors):
    return normalize(vectors)

# Read and prepare the data
ids, vectors, metadata = read_csv("synthetic-order-data.csv")
vectors = normalize_vectors(vectors)

# Prepare the data for upsert: (id, vector, metadata)
data = [(id, vector.tolist(), {'metadata': meta}) for id, vector, meta in zip(ids, vectors, metadata)]

# Connect to the Pinecone index
index = pc.Index(index_name)

# Upsert the data in batches

#index.upsert(vectors=data)
batch_size = 100  # Set the batch size to 1000
for i in range(0, len(data), batch_size):
    batch = data[i : i + batch_size]
    index.upsert(vectors=batch)
    print(f"Upserted batch {i // batch_size + 1} of {len(data) // batch_size + 1}")

# Verify success
print(f"Successfully upserted {len(data)} vectors into Pinecone.")


Upserted batch 1 of 25
Upserted batch 2 of 25
Upserted batch 3 of 25
Upserted batch 4 of 25
Upserted batch 5 of 25
Upserted batch 6 of 25
Upserted batch 7 of 25
Upserted batch 8 of 25
Upserted batch 9 of 25
Upserted batch 10 of 25
Upserted batch 11 of 25
Upserted batch 12 of 25
Upserted batch 13 of 25
Upserted batch 14 of 25
Upserted batch 15 of 25
Upserted batch 16 of 25
Upserted batch 17 of 25
Upserted batch 18 of 25
Upserted batch 19 of 25
Upserted batch 20 of 25
Upserted batch 21 of 25
Upserted batch 22 of 25
Upserted batch 23 of 25
Upserted batch 24 of 25
Upserted batch 25 of 25
Successfully upserted 2475 vectors into Pinecone.
